<a href="https://colab.research.google.com/github/eileenblum/Lyric-Generator/blob/main/Metal_data_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import string
import nltk
import gensim
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
!pip install word2vec
import word2vec

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import abc
from nltk.corpus import stopwords
from textblob import Word
from gensim.models import Word2Vec
sns.set_style("whitegrid")


##########################  1  #################################################
############# Loading the data #################################################
################################################################################
#establish working directory:
os.chdir(r'/content/drive/MyDrive/metal_lyrics')
os.getcwd()
entries = os.scandir(r'/content/drive/MyDrive/metal_lyrics')
entries

###############  Gather the Metal Files  ################################
#Make sure to change this path to where YOU are storing lyrics in your computer    
Metal_name_and_text = {}
count = 0
# Walking a directory tree and printing the names of the directories and files
for dirpath, dirnames, files in os.walk(r'/content/drive/MyDrive/metal_lyrics'):
    #print(f'Found directory: {dirpath}')
    for file_name in files:
        count = count + 1
        os.listdir()
        if count%1000 == 0:
          print(count)
        #print("song:", file_name)
        with open(dirpath + r'/' + file_name, "r", errors='ignore') as target_file:
            Metal_name_and_text[file_name] = target_file.read()

Metal_data = (pd.DataFrame.from_dict(Metal_name_and_text, orient='index')).reset_index().rename(index=str, columns={'index': 'song_name', 0: 'lyrics'})



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [ ]:
# Getting rid of two non-song files at the beginning (lml_archive.txt and .DS_Store,
# and metal_lyrics.iml, modules.xml, workspace.xml, and profile_settings.xml at the end):

Metal_data = Metal_data[2:-4]
Metal_data.reset_index(inplace=True, drop=True)
Metal_data.head()

In [ ]:
# Adding "type" and getting rid of instrumentals.

Type = np.ones(shape=(len(Metal_data),1 ))
Metal_data['type'] = Type

for i in Metal_data.index:
  if (Metal_data.loc[i,"lyrics"] == "instrumental") or (Metal_data.loc[i,"lyrics"] == ""):
    Metal_data.drop(i,inplace=True)

Metal_data.reindex(inplace=True, drop=True)

In [ ]:
##########################  2  #################################################
############# Preprocessing the Data ###########################################
################################################################################

Lyrics = Metal_data.copy()

Lyrics = Lyrics.dropna()

################# Cleaning ##############################
stop = stopwords.words('english') #importing ENglish stop words
#cleaning and lemmatizing Lyrics:
Pro_Lyrics = Lyrics.copy()
Pro_Lyrics['lyrics'] = Pro_Lyrics['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))#All the rows of the text in the data frame is checked for string punctuations, and these are filtered
Pro_Lyrics['lyrics'] = Pro_Lyrics['lyrics'].str.replace('[^\w\s]','')#REmoves dots using regular expressions
Pro_Lyrics['lyrics'] = Pro_Lyrics['lyrics'].apply(lambda x:' '.join(x.lower() for x in x.split())) #Converting text to lower case
Pro_Lyrics['lyrics'] = Pro_Lyrics['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))#Digits are removed from the text
Pro_Lyrics['lyrics'] = Pro_Lyrics['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if not x in stop))#Stop words are removed at this stage
Pro_Lyrics['lyrics'] = Pro_Lyrics['lyrics'].apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))#Words are filtered now, and different form of the same word is removed using lemmatization

Pro_Lyrics_list = []
for i in Pro_Lyrics['lyrics']:
     li = list(i.split(" "))
     Pro_Lyrics_list.append(li)	
     
Pro_Lyrics['words'] = Pro_Lyrics_list #Adding the list of words/lemmas used for every song     

################################################################################
#######  END OF PREPROCESSING  #################################################
################################################################################

In [ ]:
for i in Pro_Lyrics.index:
  if (Pro_Lyrics.loc[i,"lyrics"] == "instrumental") or (Pro_Lyrics.loc[i,"lyrics"] == ""):
    Pro_Lyrics.drop(i,inplace=True)

Pro_Lyrics.reset_index(inplace=True, drop=True)

In [ ]:
Pro_Lyrics.sample(15)

,level_0,index,song_name,lyrics,type,words
10610,10610,11806,7. Loving You.txt,verse confused knowing used blinded headed hel...,1.0,"[verse, confused, knowing, used, blinded, head..."
15015,15015,16611,4. I ve Waited.txt,waited long alone somehow knew would find woul...,1.0,"[waited, long, alone, somehow, knew, would, fi..."
13370,13370,14790,2. Svarta Pisten.txt,snãkanoner har talat pisten ligger blank som e...,1.0,"[snãkanoner, har, talat, pisten, ligger, blank..."
48103,48103,53117,1. Metal Health.txt,well im axe grinder piledriver mother say neve...,1.0,"[well, im, axe, grinder, piledriver, mother, s..."
24063,24063,26562,4. Enfermo Poder.txt,derramando enfermedad infectando con guerras m...,1.0,"[derramando, enfermedad, infectando, con, guer..."
10012,10012,11165,14. A Dead Current.txt,hot topic edition track light heaven burning s...,1.0,"[hot, topic, edition, track, light, heaven, bu..."
36343,36343,40164,2. Crawl Back.txt,known scream known today relayed thought tranq...,1.0,"[known, scream, known, today, relayed, thought..."
49592,49592,54721,15. Leaving Traces.txt,one look last second stay forever beautiful th...,1.0,"[one, look, last, second, stay, forever, beaut..."
13869,13869,15355,4. Jesse James.txt,nation set fire war lost month spent bushwhack...,1.0,"[nation, set, fire, war, lost, month, spent, b..."
52116,52116,57542,4. Nightmare Logic.txt,slumber reason give birth demon new battle tak...,1.0,"[slumber, reason, give, birth, demon, new, bat..."


In [ ]:
import pickle

with open("/content/drive/MyDrive/NLP bootcamp project/Pro Lyrics list.txt", "wb") as fp:
  pickle.dump(Pro_Lyrics_list, fp)

Pro_Lyrics.to_csv("/content/drive/MyDrive/NLP bootcamp project/Pro_Lyrics_df.csv")
